In [ ]:
import logging
import re
import ssl
import time
from datetime import date
from functools import reduce

import numpy as np
import pandas as pd
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from slack_sdk.web.slack_response import SlackResponse
from sqlalchemy import MetaData, Table, Selectable
from sqlalchemy.engine import Engine
from sqlalchemy.sql import and_, case, func, or_, select, text
from sqlalchemy.exc import NoSuchTableError

from pax_achievements import (
    cadre,
    el_presidente,
    hammer_not_nail,
    hdtf,
    leader_of_men,
    nation_sql,
    posts,
    region_sql,
    six_pack,
    the_boss,
    the_monk,
    the_priest,
award_list,
award_view,
load_to_database
)
from utils import mysql_connection, send_to_slack


In [ ]:
year = date.today().year
engine = mysql_connection()
metadata = MetaData()
metadata.reflect(engine, schema="weaselbot")

df_regions = pd.read_sql(region_sql(metadata), engine)
nation_df = pd.read_sql(nation_sql(metadata, year), engine, parse_dates="date")

# for QSource, we want to capture only QSource
bb_filter = nation_df.backblast.str.lower().str[:100].str.contains(r"q.{0,1}source|q{0,1}[1-9]\.[0-9]\s")
ao_filter = nation_df.ao.str.contains(r"q.{0,1}source")

dfs = []
############# Q Source ##############
dfs.append(the_priest(nation_df, bb_filter, ao_filter))
dfs.append(the_monk(nation_df, bb_filter, ao_filter))
############### END #################

# For beatdowns, we want to exclude QSource and Ruck (blackops too? What is blackops?)
bb_filter = nation_df.backblast.str.lower().str[:100].str.contains(r"q.{0,1}source|q{0,1}[1-9]\.[0-9]\s|ruck")
ao_filter = nation_df.ao.str.contains(r"q.{0,1}source|ruck")

############ ALL ELSE ###############
dfs.append(leader_of_men(nation_df, bb_filter, ao_filter))
dfs.append(the_boss(nation_df, bb_filter, ao_filter))
dfs.append(hammer_not_nail(nation_df, bb_filter, ao_filter))
dfs.append(cadre(nation_df, bb_filter, ao_filter))
dfs.append(el_presidente(nation_df, bb_filter, ao_filter))

s = posts(nation_df, bb_filter, ao_filter)
for val in [25, 50, 100, 150, 200]:
    dfs.append(s[s.ao_id >= val].reset_index().drop('ao_id', axis=1).rename({"date": "date_awarded"}, axis=1))

dfs.append(six_pack(nation_df, bb_filter, ao_filter))
dfs.append(hdtf(nation_df, bb_filter, ao_filter))

In [ ]:
# for testing: f3stlmeramec
# There's what already exists in the table of accomplishments (current id and all together).
# as the loop is running, need to keep track of both. For pax x, what has been earned so far for current id and in total (including wnat's being added now).

In [ ]:
engine = mysql_connection()
for row in df_regions.itertuples(index=False):
    if row.paxminer_schema == 'f3chicago':
        try:
            awarded = pd.read_sql(award_view(row, engine, metadata, year), engine, parse_dates=['date_awarded', 'created', 'updated'])
            awards = pd.read_sql(award_list(row, engine, metadata), engine)
        except NoSuchTableError:
            logging.error(f"{row.paxminer_schema} isn't signed up for Weaselbot achievements.")
            continue
        data_to_load = send_to_slack(row, year, awarded, awards, dfs)
        if not data_to_load.empty:
            load_to_database(row, engine, metadata, data_to_load)
            print(data_to_load)
        logging.info(f"Successfully loaded all records and sent all Slack messages for {row.paxminer_schema}.")